# [Heritiana Daniel Andriasolofo](mailto:handriasolofo@aimsammi.org)

The goal of this lab is to implement a language identifier (LID).

Our first model will be based on Naive Bayes.

In [1]:
import io, sys, math, re
from collections import defaultdict

In [ ]:
!rm data.zip train* valid*
!wget -O data.zip https://github.com/heritiana-aimsammi-sn2022/NLP_week1/blob/main/Lab1/data/data.zip?raw=true
!unzip data.zip
!rm data.zip

The next function is used to load the data. Each line of the data consist of a label (corresponding to a language), followed by some text, written in that language. Here is an example of data:

```__label__de Zur Namensdeutung gibt es mehrere Varianten.```


In [3]:
def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

You can now try loading the first dataset `train1.txt` and look what examples look like.

In [4]:
data = load_data("train1.txt")
print(data[0])

('__label__de', ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.'])


Next, we will start implementing the Naive Bayes method. This technique is based on word counts, and we thus need to start by implementing a function to count the words and labels of our training set.

`n_examples` is the total number of examples

`n_words_per_label` is the total number of words for a given label

`label_counts` is the number of times a given label appears in the training data

`word_counts` is the number of times a word appears with a given label

In [5]:
def count_words(data):
    n_examples = 0
    n_words_per_label = defaultdict(lambda: 0)
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for example in data:
        label, sentence = example
        ## FILL CODE
        n_examples += 1
        n_words_per_label[label] += len(sentence)
        label_counts[label] += 1
        for word in sentence:
            word_counts[label][word] += 1

    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'n_examples': n_examples, 
            'n_words_per_label': n_words_per_label}

Next, using the word and label counts from the previous function, we can implement the prediction function.

Here, `mu` is a regularization parameter (Laplace smoothing), and `sentence` is the list of words corresponding to the test example.

In [6]:
def predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label):
    best_label = None
    best_score = float('-inf')

    for label in word_counts.keys():
        score = 0.0
        ## FILE CODE
        # To ease the code, I suppose that the number of vocabulary is 5 times of the vocabulary appear in the label.
        V = 5*len(word_counts[label].keys()) 
        log_p_label = math.log(label_counts[label]) - math.log(n_examples)
        score += log_p_label
        for word in sentence:
            log_num = math.log(mu + word_counts[label][word])
            log_denom = math.log(mu*V + n_words_per_label[label])
            log_p_word_given_label = log_num - log_denom
            score += log_p_word_given_label
        if best_score < score:
            best_score = score
            best_label = label

    return best_label

The next function will be used to evaluate the Naive Bayes model on a validation set. It computes the accuracy for a particular regularization parameter `mu`.

In [7]:
def compute_accuracy(valid_data, mu, counts):
    accuracy = 0.0
    label_counts = counts["label_counts"]
    word_counts = counts["word_counts"]
    n_examples = counts["n_examples"]
    n_words_per_label = counts["n_words_per_label"]
    for label, sentence in valid_data:
        ## FILL CODE
        label_pred = predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label)
        if label == label_pred:
            accuracy += 1

     
    return accuracy/len(valid_data)

In [8]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("train1.txt")
valid_data = load_data("valid1.txt")
counts = count_words(train_data)
print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")


** Naive Bayes **

Validation accuracy: 0.945

